## Org MADS

In [ ]:
import numpy as np

def objective(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

# Bounds for each variable
bounds = np.array([[0, 1], [10, 20], [0, 3]])

# Initial guess
x0 = np.array([0.9, 19, 0.1])

def mads(func, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
    """ Mesh Adaptive Direct Search (MADS) implementation. """
    n = len(x0)
    x = x0.copy()
    delta_min = tol
    iteration = 0
    
    while delta > delta_min and iteration < max_iter:
        improved = False
        
        # Generate trial points
        for i in range(n):
            for direction in [-1, 1]:
                x_new = x.copy()
                x_new[i] += direction * delta
                x_new = np.clip(x_new, bounds[:, 0], bounds[:, 1])  # Enforce bounds
                
                if func(x_new) < func(x):
                    x = x_new
                    improved = True
        
        # Reduce mesh size if no improvement
        if not improved:
            delta /= 2.0
        
        iteration += 1
        yield x, func(x), iteration

# Run MADS optimization
mads_iter = mads(objective, x0, bounds)
for x, fval, step in mads_iter:
    print(f"Step {step}: x = {x}, f(x) = {fval}")

### Stepwise MADS, on iterations

In [2]:
import numpy as np

class StepwiseMADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.pending_eval = None
        self.best_value = None  # Store the best evaluation value
    
    def get_next_candidate(self):
        if self.pending_eval is not None:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates
        return candidates
    
    def update_with_result(self, evaluations):
        if self.pending_eval is None:
            raise RuntimeError("No pending evaluations to update with.")
        
        best_idx = np.argmin(evaluations)
        best_candidate = self.pending_eval[best_idx]
        best_value = evaluations[best_idx]
        
        if self.best_value is None or best_value < self.best_value:
            self.x = best_candidate
            self.best_value = best_value
            self.mesh_size *= self.alpha
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = None
        self.iteration += 1
    
    def get_current_solution(self):
        return self.x

# Example Usage
def evaluate(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

bounds = np.array([[0, 1], [10, 20], [0, 3]])
mads = StepwiseMADS(x0=[0.9, 19, 0.1], bounds=bounds)
mads.best_value = evaluate(mads.x)  # Initialize best value with initial point evaluation

for _ in range(20):  # Run for a few iterations
    candidates = mads.get_next_candidate()
    evaluations = [evaluate(c) for c in candidates]  # Simulated evaluation
    mads.update_with_result(evaluations)
    print(f"Iteration {_}: Best solution {mads.get_current_solution()} with value {mads.best_value}")


Iteration 0: Best solution [ 0.9 18.   0.1] with value 11.12
Iteration 1: Best solution [ 0.9 16.   0.1] with value 3.12
Iteration 2: Best solution [ 0.9 16.   0.1] with value 3.12
Iteration 3: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 4: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 5: Best solution [ 0.9 16.   2.1] with value 1.5200000000000002
Iteration 6: Best solution [ 0.9 15.   2.1] with value 0.5200000000000001
Iteration 7: Best solution [ 0.9 15.   2.1] with value 0.5200000000000001
Iteration 8: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 9: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 10: Best solution [ 0.9 15.   1.1] with value 0.31999999999999995
Iteration 11: Best solution [ 0.4 15.   1.1] with value 0.16999999999999993
Iteration 12: Best solution [ 0.4 15.   1.1] with value 0.16999999999999993
Iteration 13: Best solution [ 0.4 15.   1.6] with value 0.0200000000000